In [1]:
import pandas as pd
pd.set_option("display.float_format", lambda x: "%.3f" % x)
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True) 

from sklearn.cluster import KMeans
color = sns.color_palette()

from IPython.core.display import display, HTML 
display(HTML("{ width:100% !important; }"))
%matplotlib inline

In [4]:
df = pd.read_csv("Housing.csv")
#df

In [6]:
df.describe()

,RegionID,SizeRank,RegionName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,2000-06-30,2000-07-31,...,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30
count,27339.000,27339.000,27339.000,13489.000,13579.000,13595.000,13622.000,13689.000,13710.000,13723.000,...,26921.000,26972.000,27036.000,27064.000,27090.000,27091.000,27176.000,27193.000,27257.000,27332.000
mean,80477.882,14392.359,48723.529,154473.301,154766.438,155239.430,156342.172,157520.516,158631.812,159797.297,...,285572.372,290045.131,293738.542,296793.204,299631.321,303443.412,307336.851,311910.595,315930.381,320075.912
std,26006.407,8742.151,27433.984,115853.572,116318.670,117092.769,118726.921,120836.789,122605.980,124353.516,...,287133.884,292290.986,295822.415,298254.189,300932.196,304023.050,307648.995,312291.203,318032.937,323922.229
min,58196.000,0.000,1001.000,9417.000,9762.000,10033.000,10663.000,11208.000,11693.000,12047.000,...,14264.000,14383.000,14557.000,14687.000,14759.000,15133.000,15506.000,15855.000,15915.000,15995.000
25%,68985.000,6900.500,26282.500,86786.000,86800.500,86951.000,87252.250,87621.000,87981.000,88356.000,...,132243.000,133670.000,135013.500,136092.250,136918.500,138997.500,141132.500,143323.000,144389.000,145348.000
50%,79012.000,13888.000,48091.000,127646.000,127934.000,128139.000,128776.500,129435.000,129905.000,130537.000,...,206734.000,209534.000,212339.500,215160.500,217306.500,220384.000,223411.500,226808.000,228971.000,231152.000
75%,89258.500,21520.000,71751.500,185683.000,185890.000,186415.500,187485.250,188728.000,190148.750,191529.500,...,337022.000,342908.750,347889.250,352672.000,356451.000,361348.000,366401.750,371448.000,375843.000,380622.500
max,753844.000,34430.000,99901.000,2598211.000,2627048.000,2665734.000,2745102.000,2837169.000,2934390.000,3025305.000,...,7334906.000,7477563.000,7598042.000,7681873.000,7745686.000,7794389.000,7891864.000,7966192.000,8051814.000,8074774.000


In [7]:
df.rename(columns={"RegionName":"ZipCode"}, inplace=True)
df["ZipCode"]=df["ZipCode"].map(lambda x: "{:.0f}".format(x))
df["RegionID"]=df["RegionID"].map(lambda x: "{:.0f}".format(x))
df.head()

,RegionID,SizeRank,ZipCode,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,324341.000,...,1112872.000,1121726.000,1132075.000,1136464.000,1135845.000,1136415.000,1142305.000,1154055.000,1172761.000,1189420.000
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,313589.000,...,517190.000,521689.000,524281.000,526871.000,529670.000,531671.000,533059.000,533273.000,535955.000,538050.000
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,504802.000,...,1462303.000,1476686.000,1492446.000,1501073.000,1505320.000,1505633.000,1516210.000,1525925.000,1542865.000,1555988.000
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,225097.000,...,401739.000,412652.000,421798.000,429778.000,436880.000,442631.000,448741.000,458297.000,470006.000,483803.000
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,408512.000,...,659001.000,664046.000,667237.000,671157.000,675812.000,678741.000,680656.000,680418.000,683519.000,685964.000


In [8]:
median_prices = df.median()

C:\Users\HP\AppData\Local\Temp/ipykernel_5456/376284158.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  median_prices = df.median()


In [10]:
median_prices.tail()

2021-12-31   220384.000
2022-01-31   223411.500
2022-02-28   226808.000
2022-03-31   228971.000
2022-04-30   231152.000
dtype: float64

In [17]:
#sf_prices = df["RegionName"] == "San Francisco".median()
sc_f = df[df["CountyName"] == "Santa Clara"].median()
sf_df = df[df["City"] == "San Francisco"].median()
los_ang= df[df["City"] == "Palo Alto"].median()
df_comparison = pd.concat([marin_df, sf_df, palo_alto, median_prices], axis=1)
df_comparison.columns = ["Marin County", "San Francisco", "Palo Alto", "Median USA"]
import cufflinks as cf
cf.go_offline()
df_comparison.iplot(title="Bay Area Median Single Family Home Prices 2000-2022",
                    xTitle="Year",
                    yTitle="Sales Price",
                   #bestfit=True, bestfit_colors=["pink"],
                   #subplots=True,
                   shape=(4,1),
                    #subplot_titles=True,
                    fill=True,)

C:\Users\HP\AppData\Local\Temp/ipykernel_5456/3629507379.py:3: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.

C:\Users\HP\AppData\Local\Temp/ipykernel_5456/3629507379.py:4: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



In [18]:
marin_df = df[df["CountyName"] == "Marin"].median()
sf_df = df[df["City"] == "San Francisco"].median()
palo_alto = df[df["City"] == "Palo Alto"].median()
df_comparison = pd.concat([marin_df, sf_df, palo_alto, median_prices], axis=1)
df_comparison.columns = ["Marin County","San Francisco", "Palo Alto", "Median USA"]

C:\Users\HP\AppData\Local\Temp/ipykernel_5456/1872928650.py:2: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.

C:\Users\HP\AppData\Local\Temp/ipykernel_5456/1872928650.py:3: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



In [25]:
ca_df = df[df["StateName"] == "CA"].median()
sf_df = df[df["CountyName"] == "San Francisco"].median()
los_ang = df[df["CountyName"] == "Los Angeles"].median()
df_comparison = pd.concat([ca_df, sf_df, los_ang, median_prices], axis=1)
df_comparison.columns = ["CA state","San Francisco", "Los Angeles", "Median USA"]

C:\Users\HP\AppData\Local\Temp/ipykernel_5456/3653745613.py:1: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



In [28]:
import cufflinks as cf
cf.go_offline()
df_comparison.iplot(title="CA MedianSingle Family Home Prices 2000-2022", 
    xTitle="Year", yTitle="Sales Price",#bestfit=True, bestfit_colors=["pink"],
    #subplots=True,
    shape=(4,1),
    #subplot_titles=True, fill=True,)
    fill=True)

In [20]:
cf.go_offline()
df_comparison.iplot(title="Bay Area MedianSingle Family Home Prices 2000-2022", 
    xTitle="Year",yTitle="Sales Price",#bestfit=True, bestfit_colors=["pink"],
    #subplots=True,
    shape=(4,1),
    #subplot_titles=True, fill=True,)
    fill=True)

In [22]:
NY_df = df[df["StateName"] == "NY"].median()
nyc_df = df[df["City"] == "NY"].median()
#palo_alto = df[df["City"] == "Palo Alto"].median()
df_comparison = pd.concat([NY_df, nyc_df, median_prices], axis=1)
df_comparison.columns = ["NY State","New York", "Median USA"]

C:\Users\HP\AppData\Local\Temp/ipykernel_5456/66171252.py:1: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



In [23]:
cf.go_offline()
df_comparison.iplot(title="New York MedianSingle Family Home Prices 2000-2022", 
    xTitle="Year",yTitle="Sales Price",#bestfit=True, bestfit_colors=["pink"],
    #subplots=True,
    shape=(4,1),
    #subplot_titles=True, fill=True,)
    fill=True)

In [40]:
#Cluster on Size Rank and Price
from sklearn.preprocessing import MinMaxScaler
columns_to_drop = ['RegionID', 'ZipCode', 'City', 'State', 'Metro', 'CountyName','RegionType', 'StateName']
df_numerical = df.dropna()
df_numerical = df_numerical.drop(columns_to_drop, axis=1)
df_numerical.describe()

,SizeRank,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,2000-06-30,2000-07-31,2000-08-31,2000-09-30,...,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30
count,9520.000,9520.000,9520.000,9520.000,9520.000,9520.000,9520.000,9520.000,9520.000,9520.000,...,9520.000,9520.000,9520.000,9520.000,9520.000,9520.000,9520.000,9520.000,9520.000,9520.000
mean,9363.613,169123.017,169739.046,170370.277,171694.384,173007.987,174345.834,175687.696,177135.459,178622.350,...,414673.886,421953.718,427590.693,432031.711,436434.532,441315.459,446998.540,453598.289,461162.849,469074.410
std,7090.340,120153.672,120904.209,121779.754,123617.930,125528.648,127506.702,129474.867,131556.859,133695.269,...,362861.716,369434.416,373863.844,376636.675,379809.627,383503.465,388342.352,394330.607,402005.924,409773.325
min,3.000,9417.000,9762.000,10033.000,10663.000,11208.000,11693.000,12047.000,12370.000,12617.000,...,23475.000,23243.000,23143.000,23407.000,23939.000,24680.000,25440.000,26809.000,28105.000,28435.000
25%,3592.500,99590.500,99842.000,100079.250,100570.000,100912.250,101364.500,101643.500,102179.500,102531.000,...,207403.750,211211.000,214152.500,217090.750,219971.000,223313.500,226559.500,230150.250,233346.000,236999.250
50%,7788.000,142323.500,142779.000,143231.000,144092.500,144964.000,145701.500,146329.500,146978.500,148013.500,...,321986.000,327895.500,332644.500,336479.000,340811.000,345639.500,350638.000,355915.000,361316.000,367559.500
75%,13824.000,201702.750,202373.000,202848.250,204332.250,205877.000,207005.250,208577.750,210049.250,211718.250,...,499137.000,508869.500,515447.000,521334.000,527207.250,533943.500,540380.750,547519.000,556443.250,565854.500
max,34430.000,2598211.000,2627048.000,2665734.000,2745102.000,2837169.000,2934390.000,3025305.000,3106879.000,3191488.000,...,7334906.000,7477563.000,7598042.000,7681873.000,7745686.000,7794389.000,7891864.000,7966192.000,8051814.000,8074774.000


In [41]:
df_numerical

,SizeRank,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,2000-06-30,2000-07-31,2000-08-31,2000-09-30,...,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30
3,3,225097.000,225416.000,226142.000,227372.000,226933.000,226596.000,225474.000,225817.000,225418.000,...,401739.000,412652.000,421798.000,429778.000,436880.000,442631.000,448741.000,458297.000,470006.000,483803.000
5,5,106812.000,106825.000,106628.000,106633.000,106664.000,106876.000,107030.000,107258.000,107640.000,...,228997.000,235330.000,240408.000,244986.000,248336.000,252261.000,256197.000,260849.000,266524.000,272800.000
6,6,409351.000,403981.000,404189.000,402311.000,407324.000,410228.000,414704.000,415460.000,415058.000,...,871367.000,899999.000,909282.000,912446.000,918043.000,921182.000,926304.000,926163.000,930629.000,937082.000
7,7,106328.000,106273.000,106016.000,105964.000,105930.000,106096.000,106136.000,106322.000,106722.000,...,226657.000,232971.000,237966.000,242382.000,245339.000,248814.000,252235.000,256378.000,261612.000,267377.000
8,8,90610.000,90610.000,90643.000,90609.000,90628.000,90588.000,90573.000,90526.000,89994.000,...,162913.000,165717.000,168612.000,170924.000,172889.000,174512.000,176526.000,178504.000,180662.000,184033.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27310,33487,283986.000,283502.000,286100.000,289885.000,293778.000,297077.000,302631.000,310736.000,317997.000,...,972122.000,1030448.000,1069976.000,1085022.000,1094538.000,1105181.000,1121778.000,1129990.000,1130952.000,1129584.000
27313,33581,205652.000,206110.000,206667.000,209776.000,213670.000,216877.000,216560.000,209947.000,203834.000,...,394248.000,398863.000,402321.000,403970.000,407060.000,408740.000,412047.000,415461.000,418848.000,421690.000
27331,34322,135840.000,136537.000,137563.000,137586.000,137970.000,139003.000,141185.000,144623.000,147062.000,...,340738.000,347054.000,352640.000,354984.000,359656.000,361640.000,367410.000,374308.000,382069.000,393807.000
27335,34430,486003.000,486258.000,489144.000,491714.000,493964.000,496664.000,502387.000,506897.000,509005.000,...,1161337.000,1174731.000,1183444.000,1191437.000,1206578.000,1221541.000,1253566.000,1281109.000,1304270.000,1315532.000


In [42]:
scaler = MinMaxScaler()
scaled_df = scaler.fit_transform(df_numerical)
kmeans = KMeans(n_clusters=3, random_state=0).fit(scaled_df)
print(len(kmeans.labels_))

9520


In [44]:
cluster_df = df.copy(deep=True)
cluster_df.dropna(inplace=True)
cluster_df.describe()
cluster_df['cluster'] = kmeans.labels_
cluster_df['appreciation_ratio'] = round(cluster_df["2017-09"]/cluster_df["1996-04"],2)
cluster_df['CityZipCodeAppRatio'] = cluster_df['City'].map(str) + "-" + cluster_df['ZipCode'] + "-" + cluster_df["appreciation_ratio"].map(str)
cluster_df.head()

KeyError: '2017-09'